In [1]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap
import networkx as nx
import queue

In [2]:
class Agent:
    pos = [0,0]
    direction = 0
    size = 3
    def __init__(self,x,y,direction):
        self.pos = [x,y]
        self.direction = direction
        
    def getSize(self):
        return self.size
    
    def getPos(self):
        return self.pos
    
    def getDirection(self):
        return self.direction
    
    def getCarFront(self):
        #0=N, 2=E, 4=S, 6=W
        front = self.pos.copy()
        if self.direction == 0:
            front[1] -= 1
        elif self.direction == 1:
            front[0] += 1
            front[1] -= 1
        elif self.direction == 2:
            front[0] += 1
        elif self.direction == 3:
            front[0] += 1
            front[1] += 1
        elif self.direction == 4:
            front[1] += 1
        elif self.direction == 5:
            front[0] -= 1
            front[1] += 1
        elif self.direction == 6:
            front[0] -= 1
        elif self.direction == 7:
            front[0] -= 1
            front[1] -= 1
        
        return front
        
    def turn(self, direction):
        self.direction = (self.direction + direction) % 8
        #Example case, turn left from North
        if self.direction < 0:
            self.direction = 8 + self.direction
    
    def moveForward(self, steps):
        #0=N, 2=E, 4=S, 6=W
        if self.direction == 0:
            self.pos[1] -= steps
        elif self.direction == 1:
            self.pos[0] += steps
            self.pos[1] -= steps
        elif self.direction == 2:
            self.pos[0] += steps
        elif self.direction == 3:
            self.pos[0] += steps
            self.pos[1] += steps
        elif self.direction == 4:
            self.pos[1] += steps
        elif self.direction == 5:
            self.pos[0] -= steps
            self.pos[1] += steps
        elif self.direction == 6:
            self.pos[0] -= steps
        elif self.direction == 7:
            self.pos[0] -= steps
            self.pos[1] -= steps
            


In [3]:
class Obstacle:
    pos = [0,0]
    face = 0
    size = 1
    
    def __init__(self, *args):
        if len(args) == 2:
            self.pos = [args[0], args[1]]
        elif len(args) == 3:
            self.pos = [args[0], args[1]]
            self.face = args[2]
        else:
            print("Incorrect argument count")
                
                
    def __eq__(self, other):
        if self.pos == other.pos and self.face == other.face:
            return True
        return False
    
    def getObsFacing(self):
        return self.face

In [4]:
class Map:
    mapGrid = []
    car = Agent(0,0,0)
    obsList = []
    obsDirDict = {}
    borders = set()
    size = 0
    
    def __init__(self,size,x,y,direction):
        self.size = size + 2
        for i in range(self.size):
            self.borders.add((0,i))
            self.borders.add((self.size-1, i))
            self.borders.add((i,0))
            self.borders.add((i,self.size-1))
        #print(self.borders)
        self.initMap(self.size,x,y,direction)
    
    def initMap(self,size,x,y,direction):
        self.car.pos = [x,y]
        self.car.direction = direction
        for i in range(self.size):
            templist= []
            for j in range(self.size):
                if ((i,j) in self.borders):
                    templist.append("W")
                else:
                    templist.append("-")
            self.mapGrid.append(templist)
        self.markCar()
    
    def printMap(self):
        for i in range(self.size):
            for j in range(self.size):
                print(self.mapGrid[j][i], end = " ")
            print()
            
    def resetCar(self):
        loc = self.car.getPos()
        sizeCar = self.car.getSize()
        #wingspan of the car
        span = int((sizeCar - 1)/2)
        
        #Remove car on map
        for i in range(-span, span+1):
            for j in range(-span, span+1):
                    self.mapGrid[loc[0]+i][loc[1]+j] = '-'
             
    
    def markCar(self):
        loc = self.car.getPos()
        front = self.car.getCarFront()
        sizeCar = self.car.getSize()
        #wingspan of the car
        span = int((sizeCar - 1)/2)
        
        #Avoid checking more if one fails
        fail = 0
        #Mark car on map
        for i in range(-span, span+1):
            for j in range(-span, span+1):
                #Check whether car is in border or in Obstacle
                if (loc[0]+i>=0 and loc[0]+i<= self.size and loc[1]+j>=0 
                    and loc[1]+j<= self.size and (loc[0]+i,loc[1]+j) not in self.borders 
                    and Obstacle(loc[0]+i,loc[1]+j) not in self.obsList):
                     
                    if i == 0 and j == 0:
                        self.mapGrid[loc[0]][loc[1]] = '$'
                    elif (loc[0]+i) == front[0] and (loc[1]+j) == front[1]:
                        self.mapGrid[loc[0]+i][loc[1]+j] = '@'
                    else:
                        self.mapGrid[loc[0]+i][loc[1]+j] = '+'
                #Remove car if in border
                else:
                    fail = 1
                    break
            
            if fail == 1:
                    self.checkCarBorder()
                    print("Car in border obstacle")
                    break
    
    
    def checkCarBorder(self):
        loc = self.car.getPos()
        sizeCar = self.car.getSize()
        #wingspan of the car
        span = int((sizeCar - 1)/2)
        
        #Remove car on map
        for i in range(-span, span+1):
            for j in range(-span, span+1):
                #Check whether car is in border
                if (loc[0]+i>=0 and loc[0]+i<=self.size and loc[1]+j>=0 
                    and loc[1]+j<=self.size):
                    if (loc[0]+i,loc[1]+j) in self.borders:
                        self.mapGrid[loc[0]+i][loc[1]+j] = 'W'
                    elif Obstacle(loc[0]+i,loc[1]+j) in self.obsList:
                        self.mapGrid[loc[0]+i][loc[1]+j] = self.obsDirDict[(loc[0]+i,loc[1]+j)]
                    else:
                        self.mapGrid[loc[0]+i][loc[1]+j] = '-'
                    
    def moveCar(self,steps):
        self.resetCar()
        self.car.moveForward(steps)
        self.markCar()
        
    def turnCar(self,rotate):
        self.resetCar()
        self.car.turn(rotate)
        self.markCar()
        
    def addObstacle(self,x,y,facing):
        if (x,y) not in self.borders:
            self.obsList.append(Obstacle(x,y))
            self.obsDirDict[(x,y)] = facing
            #print(self.obsDirDict)
            self.mapGrid[x][y] = facing
        else:
            print("Unable to add obstacle due to borders")

# Consider using dictionary to collate key-value pair for coordinates and direction?

In [5]:
maze = Map(20,2,19,0)

In [6]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W + @ + - - - - - - - - - - - - - - - - - W 
W + $ + - - - - - - - - - - - - - - - - - W 
W + + + - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [7]:
maze.moveCar(2)

In [8]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W + @ + - - - - - - - - - - - - - - - - - W 
W + $ + - - - - - - - - - - - - - - - - - W 
W + + + - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [9]:
maze.turnCar(1)

In [10]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W + + @ - - - - - - - - - - - - - - - - - W 
W + $ + - - - - - - - - - - - - - - - - - W 
W + + + - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [11]:
maze.moveCar(2)

In [12]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - + + @ - - - - - - - - - - - - - - - W 
W - - + $ + - - - - - - - - - - - - - - - W 
W - - + + + - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [13]:
maze.addObstacle(2,2,2)

In [14]:
maze.addObstacle(5,5,4)

In [15]:
print(Obstacle(2,2) in maze.obsList)

True


In [16]:
maze.addObstacle(0,0,4)

Unable to add obstacle due to borders


In [17]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - 2 - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - 4 - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - + + @ - - - - - - - - - - - - - - - W 
W - - + $ + - - - - - - - - - - - - - - - W 
W - - + + + - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


### Will need to account for size of wall

In [18]:
maze.turnCar(-1)

In [19]:
maze.moveCar(8)

In [20]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - 2 - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - 4 - - - - - - - - - - - - - - - W 
W - - + @ + - - - - - - - - - - - - - - - W 
W - - + $ + - - - - - - - - - - - - - - - W 
W - - + + + - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 


In [21]:
maze.moveCar(1)

Car in border obstacle


In [22]:
maze.printMap()

W W W W W W W W W W W W W W W W W W W W W W 
W - - - - - - - - - - - - - - - - - - - - W 
W - 2 - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - 4 - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W - - - - - - - - - - - - - - - - - - - - W 
W W W W W W W W W W W W W W W W W W W W W W 
